In [1]:
import string
import networkx as nx
import re
from nltk import word_tokenize, pos_tag
from pybabelfy.babelfy import *
from nltk.stem import PorterStemmer
from math import log
import pickle
import glob
import os
import OClustR as OCR
import operator
#import nltk

In [2]:
key = ''
inputFile = ''
outputDirectory = ''
distance_window = 0
graphName = []
dictionaries = []
dictionariesCode = []
graphsI = []
graphsD = []
sciKGraph = 0
pre_processed_graph = 0
dictionaryCodeMerged = {}
language = ''
deleted_nodes = 0
deleted_edges = 0
deleted_isolated_nodes = 0
name=""
clusters = []
crisp_clusters = []
pre_processed_graph = nx.DiGraph()

In [3]:
#rank Concepts
def rank(g, dictionaryCodeMerged):
        grau = nx.degree_centrality(g)
        sorted_grau = sorted(grau.items(), key=operator.itemgetter(1), reverse=True)
        sorted_concepts = []
        for i in sorted_grau:
            sorted_concepts.append(dictionaryCodeMerged[i[0]].lower().replace('+', ' ') + '  :  ' + i[0])

        return sorted_concepts

In [4]:
#key Concepts
def key_concepts( g, dictionaryCodeMerged):
    grau = nx.degree_centrality(g)
    sorted_grau = sorted(grau.items(), key=operator.itemgetter(1), reverse=True)
    sorted_concepts = []
    for i in sorted_grau:
        sorted_concepts.append([dictionaryCodeMerged[i[0]], i[1]])

    return sorted_concepts

In [5]:
# open and close file
def open_file(fileName):
    file = open(fileName,"r")
    text = file.read()
    file.close()
    return text

In [7]:
#parse and split text in chuncks of at most 3000 characters
def parse_text(text):
    #remove special characters
    punctuationToRemove = string.punctuation.replace('!','').replace('.','').replace('?','').replace('-','').replace(',','')
    translator = str.maketrans('', '', punctuationToRemove)
    parsedText = text.translate(translator)
    #remove numbers
    parsedText = re.sub(r'[0-9]+', '', parsedText)
    #remove double spaces
    parsedText = re.sub(r'  ', ' ', parsedText)
    #remove non-printable characters
    parsedText = "".join(filter(lambda x: x in string.printable, parsedText))
    #remove spaces
    parsedText = re.sub(r' ', '+', parsedText)
    #split text in chuncks of at most 5000 characters
    punctuation = ['.','?','!']
    splitted_text = []
    splitted_text.append("")
    n_lines = len(parsedText.splitlines())
    for line in parsedText.splitlines():
        if n_lines == 1:
            splitted_text[-1] = line
        else:
            if len(splitted_text[-1] + line) < 4500 and splitted_text[-1][-1:] not in punctuation or len(splitted_text[-1] + line) <= 3000:
                splitted_text[-1] = splitted_text[-1] + '+' + line
            else:
                splitted_text.append(line)
    translator = str.maketrans('', '', "?!.")
    for l in splitted_text:
        l = l.translate(translator)
    return splitted_text

In [9]:
def frag(semantic_annotation, input_text):
    start = semantic_annotation.char_fragment_start()
    end = semantic_annotation.char_fragment_end()
    return input_text[start:end+1]

In [10]:
def babelfy(lang, key, splitted_text):
    babelapi = Babelfy()
    paragraphs_annotations = []
    paragraphs_text = []
    paragraphs_code = []

    count = 0
    for paragraph in splitted_text: #annotate each paragraph
        words_annotations = []
        words_text = []
        words_code = []
        semantic_annotations = babelapi.disambiguate(paragraph,lang,key,match="EXACT_MATCHING",cands="TOP",mcs="ON",anntype="ALL")

        #exclude unused annotations (single words of multiword expressions)
        for semantic_annotation in semantic_annotations:
            if len(words_annotations) == 0 or words_annotations[-1].char_fragment_end() < semantic_annotation.char_fragment_start():
                words_annotations.append(semantic_annotation)
                words_text.append(frag(semantic_annotation,paragraph))
                words_code.append(semantic_annotation.babel_synset_id())

            elif words_annotations[-1].char_fragment_start() == semantic_annotation.char_fragment_start():
                del words_annotations[-1]
                words_annotations.append(semantic_annotation)
                del words_text[-1]
                words_text.append(frag(semantic_annotation,paragraph))
                del words_code[-1]
                words_code.append(semantic_annotation.babel_synset_id())


        paragraphs_annotations.append(words_annotations)
        paragraphs_text.append(words_text)
        paragraphs_code.append(words_code)
        count = count + 1
        print(str(count) + '/' + str(len(splitted_text)))
    return paragraphs_annotations, paragraphs_text, paragraphs_code

In [11]:
def create_dicts(paragraphs_text, paragraphs_code):
    ###        dictionary[word] = code             ###
    ###        dictionaryCode[code] = word        ###
    ###        weight[code] = weight            ###
    dictionary={}
    weight={}
    dictionaryCode={}
    for paragraph, codes in zip(paragraphs_text, paragraphs_code):
        for word, code in zip(paragraph, codes):
            if code not in weight:
                weight[code] = 1
            else:
                weight[code] = weight[code] + 1

            if word not in dictionary:
                dictionary[word] = code
            if code not in dictionaryCode:
                dictionaryCode[code] = word
    return dictionary, dictionaryCode, weight

In [12]:
def create_simple_graph(peso, paragraphs_code, dictionaryCode, dist):
    g = nx.DiGraph() #indirect Graph
    g2 = nx.DiGraph() #direct Grap

    #calc the weight of each vertice
    for code, weight in peso.items():
        g.add_node(code, peso=weight, dicionario=dictionaryCode[code])
        g2.add_node(code, peso=weight, dicionario=dictionaryCode[code])

    #create and weight edges
    for line in paragraphs_code:
        i = 0
        for word in line:
            i = i + 1
            j = 0
            for word2 in line:
                j = j + 1
                if j - i < dist and j - i > 0: #indirect edges
                    if g.has_edge(word, word2):
                        g[word][word2]['weight'] += 1 - log(j-i,dist)
                    else:
                        if word != word2:
                            g.add_edge(word, word2, weight=float(1 - log(j-i,dist)))
                if j - i == 1: #direct edges
                    if g2.has_edge(word, word2):
                        g2[word][word2]['weight'] += 1
                    else:
                        if word != word2:
                            g2.add_edge(word, word2, weight=1)
    return g, g2

In [13]:
def save_clusters_txt(saveFile, Clusters):
    f=open(saveFile,"w+")
    for c in Clusters:
        line = ''
        for n in c:
            line += n + ' '
        f.write(line[:-1] + '\n')
    f.close()
    return

In [15]:
def saveClusters(saveFile="", Clusters=[], crisp="", clusterType='normal'):
    file = ''
    #save clusters

    #write crisp
    if crisp != "":
        with open(saveFile + "crisp.pickle", "wb") as fp:
            pickle.dump(crisp, fp, protocol=2)

        f=open(saveFile + "crisp.txt","w+")
        for c in crisp:
            line = ''
            for n in c:
                line += n + ' '
            f.write(line[:-1] + '\n')
        f.close()

    #write normal clusters
    if clusterType =='normal':
        with open(saveFile + "clusters.pickle", "wb") as fp:
            pickle.dump(Clusters, fp, protocol=2)

        f=open(saveFile + "clusters.txt","w+")
        for c in Clusters:
            line = ''
            for n in c:
                line += n + ' '
            f.write(line[:-1] + '\n')
        f.close()

    #write reduced clusters
    elif clusterType =='reduced':
        with open(saveFile + "reducedClusters.pickle", "wb") as fp:
            pickle.dump(Clusters, fp, protocol=2)

        f=open(saveFile + "reducedClusters.txt","w+")
        for c in Clusters:
            line = ''
            for n in c:
                line += n + ' '
            f.write(line[:-1] + '\n')
        f.close()

    else:
        print('Wrong cluster Type!\nCluster not saved')

In [16]:
def save_variables_pickle():
    save = []
    save.append(graphName)
    save.append(dictionaries)
    save.append(dictionariesCode)
    save.append(graphsI)
    save.append(graphsD)
    save.append(dictionaryCodeMerged)
    save.append(sciKGraph)
    save.append(crisp_clusters)
    save.append(pre_processed_graph)
    save.append(clusters)

    file = pickle.dumps(save, protocol=2)
    #with open('/home/mauro/Downloads/testeDownload.sckg', "wb") as fp:
    #    pickle.dump(save, fp, protocol=2)


    return file

In [17]:
def save_variables(output_file, save_graph_name=False, save_directories = False, save_directories_code = False, save_graphs_i = False, save_graphs_d = False, save_directories_code_merged = False, save_SciKGraph = False, save_clusters = False, save_crisp_clusters = False, save_pre_processed_graph = False):
    save = []
    save.append(graphName)
    save.append(dictionaries)
    save.append(dictionariesCode)
    save.append(graphsI)
    save.append(graphsD)
    save.append(dictionaryCodeMerged)
    save.append(sciKGraph)
    save.append(crisp_clusters)
    save.append(pre_processed_graph)
    save.append(clusters)


    try:
        with open(output_file, "wb") as fp:
            pickle.dump(save, fp, protocol=2)
    except:
        raise

    return

In [18]:
def open_variables_pickle(file):
    data = pickle.load(file)

    graphName = data[0]
    dictionaries = data[1]
    dictionariesCode = data[2]
    graphsI = data[3]
    graphsD = data[4]
    dictionaryCodeMerged = data[5]
    sciKGraph = data[6]
    crisp_clusters = data[7]
    pre_processed_graph = data[8]
    clusters = data[9]

In [19]:
def open_variables(open_directory, open_graph_name=False, open_directories = False, open_directories_code = False, open_graph_i = False, open_graph_d = False, open_dictionary_code_merged = False, open_SciKGraph = False, open_clusters = False, open_crisp_clusters = False, open_pre_processed_graph = False):

    with open(open_directory, "rb") as fp:
        data = pickle.load(fp)

    graphName = data[0]
    dictionaries = data[1]
    dictionariesCode = data[2]
    graphsI = data[3]
    graphsD = data[4]
    dictionaryCodeMerged = data[5]
    sciKGraph = data[6]
    crisp_clusters = data[7]
    pre_processed_graph = data[8]
    clusters = data[9]

    return

In [20]:
def clear_variables():
    key = ''
    inputFile = ''
    outputDirectory = ''
    distance_window = 0
    graphName = []
    dictionaries = []
    dictionariesCode = []
    graphsI = []
    graphsD = []
    sciKGraph = 0
    dictionaryCodeMerged = {}
    name=""
    return

In [21]:
def create_single_SciKGraph(filename, babelfy_key, language, distance_window):
    text = filename.decode('ascii')
    st = parse_text(text)
    pa, pt, pc  = babelfy(language, babelfy_key, st)
    d, dc, p = create_dicts(pt, pc)

    gI, gD = create_simple_graph(p, pc, dc, distance_window)
    return d, dc, gI, gD

In [22]:
#Merges graphs and dictionaries
## graphs: list of graphs to merge
## dictionaryCode: list of the graphs dictionaries
def merge_graphs(graphs, dictionaryCode):
    #create dictionaryCodeMerged
    dictionaryCodeMerged = {}
    for dic in dictionaryCode:
        for w in dic:
            if w not in dictionaryCodeMerged:
                dictionaryCodeMerged[w] = dic[w]


    #merge graphs
    graph = nx.compose_all(graphs).copy()

    #reset nodes weights
    for i in graph.nodes():
        graph.nodes()[i]['peso'] = 0

    #recalc nodes weights
    for i in range(len(graphs)):
        for n in graphs[i]:
            graph.nodes()[n]['peso'] += graphs[i].nodes()[n]['peso']
            graph.nodes()[n]['dicionario'] = dictionaryCodeMerged[n]

    #reset arc weight
    for i in graph.edges():
        graph[i[0]][i[1]]['weight'] = 0

    #recalc arc weight
    for i in range(len(graphs)):
        for e in graphs[i].edges():
            graph[e[0]][e[1]]['weight'] += graphs[i][e[0]][e[1]]['weight']



    return graph, dictionaryCodeMerged

In [23]:
def create_SciKGraph(files, file_names, babelfy_key = None, language = 'EN', graphType = 'direct', distance_window=2, mergeIfFail = False):
    distance_window = distance_window + 1
    if distance_window <=2:
        graphType = 'direct'
    else:
        graphType = 'indirect'

    language = language

    #check if scikgraph should be fully updated (occurs when distance window changes)
    if distance_window != distance_window:
        distance_window = distance_window
        graphName = []

    toMerge = []
    count = 0
    added = 0
    for file, file_name in zip(files, file_names):
        count += 1
        if file_name not in graphName:
            try:
                d, dc, gI, gD = create_single_SciKGraph(file, babelfy_key, language, distance_window)

                graphName.append(file_name)
                dictionaries.append(d)
                dictionariesCode.append(dc)
                graphsI.append(gI)
                graphsD.append(gD)
                added += 1

            except Exception as e:
                if len(graphName) > 0 or mergeIfFail:
                    print('Error Babelfying text (check your Babelcoins)\n', e, '\n')
                    print(graphName, '\nThe documents in \'graphName\' were correctly babelfied.\nThe SciKGraph was created with the correctly babelfied texts, to update this version with the other texts fix the error (probably babelfy key error) and run this method again.')
                    break
                else:
                    if len(graphName) > 0:
                        print(graphName, '\nThe documents in \'graphName\' were correctly babelfied.\nTo create the SciKGraph (using the previously babelfied documents) run this method again.\n')
                    print('Error Babelfying text (check your Babelcoins)\n')
                    raise

        if graphType == 'direct':
            toMerge = graphsD
        elif graphType == 'indirect':
            toMerge = graphsI
        else:
            print('graphType not listed!\nDirect graph used.')
            toMerge = graphsD

    #check if at leat 1 graph can be added to scikgraph
    if added > 0:
        graph, dictionaryMerged = merge_graphs(toMerge, dictionariesCode)
        sciKGraph = graph
        dictionaryCodeMerged = dictionaryMerged

    return sciKGraph, dictionaryCodeMerged



In [24]:
def find_communities(g, edges_threshold, nodes_threshold):
    ocr = OCR.OClustR()
    clusters, crisp_clusters, pre_processed_graph = ocr.identify_clusters(g, edges_threshold, nodes_threshold)
    return clusters, crisp_clusters, pre_processed_graph



In [25]:
def cluster_graph(g):
    ocr = OCR.OClustR()
    clusters, crisp_clusters, sciKGraph = ocr.cluster_graph(g)
    return

In [27]:
def pre_process_graph(g, edges_threshold, nodes_threshold, list_edges = [], list_nodes = []):
    oClustR = OCR.OClustR()
    g, rem_e, rem_n, rem_iso_n = oClustR.pre_process(g, edges_threshold, nodes_threshold, list_edges, list_nodes)
    pre_processed_graph = g
    deleted_isolated_nodes = rem_iso_n
    deleted_nodes = rem_n
    deleted_edges = rem_e
    return

In [28]:
def to_crisp(Clusters):
    ##Crisp Cluster
    crisp = []
    elem = []
    for c in Clusters:
        cl = []
        for v in c:
            if v not in elem:
                cl.append(v)
                elem.append(v)
        if len(cl) >= 1:
            crisp.append(cl)
    return crisp

In [30]:
def start( inputDirectory, babelfy_key, edges_threshold=0, nodes_threshold=0, list_nodes = [], list_edges = [], language = 'EN', graphType = 'direct', distance_window=2, mergeIfFail = False):
    if babelfy_key == None:
        babelfy_key = key

    filenames = []

    try:
        for filename in sorted(glob.glob(os.path.join(inputDirectory, '*.txt'))):
            filenames.append(filename)
        if len(filename) == 0:
            raise EmptyDirectoryError('There is no .txt file in the inputDirectory.')
    except:
        raise


    sciKGraph, dictionaryCodeMerged = create_SciKGraph(filenames, babelfy_key, language, graphType, distance_window, mergeIfFail)
    return sciKGraph

In [ ]:
############### Create SciKGraph #################

In [ ]:
#documentsList (list of files)= list of documents
#documentsNamesList (list of strings)= list of the names of the documents in documentsList
#babelfy_key (string)= babelfy key 
#language (string)= 'EN'
#distance_window (int) = value of distance to consider of concepts coocorrence
#mergeIfFail (boolean) = If Babelfy fail (key problem) merged already babelfied texts?

create_SciKGraph(documentsList, documentsNamesList, babelfy_key, language, distance_window, mergeIfFail)            